In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 47.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.3.3
    Uninstalling safetensors-0.3.3:
      Successfully uninstalled safetensors-0.3.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
    

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [3]:
model_name = '/kaggle/input/mistral/pytorch/7b-v0.1-hf/1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

KeyError: 'mistral'

In [ ]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer = tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import json
import random

# Function to load the dataset from a JSON file
def load_dataset(file_path, sample_size=1000):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]

    if sample_size >= len(data):
        return data

    return random.sample(data, sample_size)

run = 3
file_path = '/kaggle/input/e-care/ecare_' + str(run) + '.json'  # Update this path
dataset = load_dataset(file_path)

In [ ]:
dataset[0]

In [ ]:
def is_response_correct(response_text, correct_hypothesis):
    response_text = response_text.lower()
    correct_hypothesis_lower = correct_hypothesis.lower()
    key_phrases = correct_hypothesis_lower.split()

    matching_phrases = [phrase for phrase in key_phrases if phrase in response_text]
    match_ratio = len(matching_phrases) / len(key_phrases)
    
    return match_ratio > 0.5  # Adjust this threshold as needed

prompt = "Which is more likely to be the effect of this - The professor dissected a cuttlefish in marine biology class. - The engineers thought they were fractures.(0). or The students learned a lot about cuttlefish's body structure.(1). Select the right statement."

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
response = sequences[0]['generated_text']
if len(response) > len(prompt):
    response = response[len(prompt):]
print(response)
print(is_response_correct(response, "The students learned a lot about cuttlefish's body structure."))

In [ ]:
import json
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Function to load the dataset from a JSON file
def load_dataset(file_path, sample_size=100000):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]

    if sample_size >= len(data):
        return data

    return random.sample(data, sample_size)

# Function to calculate F1 score 
def compute_f1_score(golds, preds):
    tp = sum(1 for g, p in zip(golds, preds) if g == 1 and p == 1)
    fp = sum(1 for g, p in zip(golds, preds) if g == 0 and p == 1)
    fn = sum(1 for g, p in zip(golds, preds) if g == 1 and p == 0)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1score

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return [accuracy, f1, precision, recall]

# Check if the response correctly aligns with a given hypothesis
def is_response_correct(response_text, correct_hypothesis):
    response_text = response_text.lower()
    correct_hypothesis_lower = correct_hypothesis.lower()
    key_phrases = correct_hypothesis_lower.split()

    matching_phrases = [phrase for phrase in key_phrases if phrase in response_text]
    match_ratio = len(matching_phrases) / len(key_phrases)

    return match_ratio > 0.5  # Adjust this threshold as needed


# Load the dataset
file_path = '/kaggle/input/e-care/ecare_' + str(run) + '.json'  # Update this path
dataset = load_dataset(file_path)

# Load Mistral model and tokenizer
model_name = '/kaggle/input/mistral/pytorch/7b-v0.1-hf/1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id
)

# File to save the generated responses
responses_file = '/kaggle/working/mistral_model_responses_' + str(run) + '.jsonl'

# Generate predictions and save responses
try:
    with open(responses_file, 'r', encoding='utf-8') as file:
        saved_responses = [json.loads(line) for line in file]
    print("Loaded saved responses.")
except FileNotFoundError:
    saved_responses = []
    for index, item in enumerate(dataset):
        print(index)
#         print(f"Item at position {index}: {item}")
        task = "cause for" if item["ask-for"] == "cause" else "effect of"
        query = f"Which is more likely to be the {task} this {item['premise']} - {item['hypothesis1']}(0). or {item['hypothesis2']}(1). Select the right statement."
        response = pipe(query, max_length=150)[0]['generated_text']
        if len(response) > len(query):
            response = response[len(query):]
        saved_responses.append({
            'index': item['index'],
            'premise': item['premise'],
            'hypothesis1': item['hypothesis1'],
            'hypothesis2': item['hypothesis2'],
            'label': item['label'],
            'generated_response': response
        })
    
    # Save the generated responses
    with open(responses_file, 'w', encoding='utf-8') as file:
        for response in saved_responses:
            file.write(json.dumps(response) + '\n')
    print("Responses saved.")

# Calculate predictions, F1 score, and accuracy
preds, golds = [], []
for item in saved_responses:
    correct_hypothesis = item['hypothesis1'] if item['label'] == 0 else item['hypothesis2']
    if is_response_correct(item['generated_response'], correct_hypothesis):
        preds.append(item['label'])
    else:
        preds.append(1 - item['label'])  # Assuming binary labels (0 and 1)
    golds.append(item['label'])

f1score = compute_f1_score(golds, preds)
accuracy = sum(1 for g, p in zip(golds, preds) if g == p) / len(golds)


print(f"F1 Score: {f1score:.2f}")
print(f"Accuracy: {accuracy:.2f}")

print('---')

sklearn_accuracy, f1val, precisionval, recallval = compute_metrics(golds, preds)
print(f"sklearn Accuracy: {sklearn_accuracy * 100:.3f}%")
print(f"F1: {f1val * 100:.3f}%")
print(f"Precision: {precisionval * 100:.3f}%")
print(f"Recall: {recallval * 100:.3f}%")

